In [31]:
import pandas as pd
import numpy as np

from scipy.stats import norm

In [32]:
t = 0
T = 0.5
sigma = 0.3
type = 'out-and-in'
K = 16
S = 15
U = 18
r = 0.12
q = 0
n = 100

dt = T / n

CALL OPTION

In [33]:
def d_plus(s, sigma, T, t, r, strike):
    return ((np.log(s / strike)) + (T - t) * (r + (sigma ** 2) / 2)) / (sigma * np.sqrt(T - t))

In [34]:
def d_minus(s, sigma, T, t, r, strike):
    return ((np.log(s / strike)) + (T - t) * (r - (sigma ** 2) / 2)) / (sigma * np.sqrt(T - t))

In [35]:
def get_call_option_price(s, sigma, T, t, r, strike):
    I_plus = norm.cdf(d_plus(s, sigma, T, t, r, strike))
    I_minus = norm.cdf(d_minus(s, sigma, T, t, r, strike))
    ans = s * I_plus - strike * np.exp(-1 * r * (T - t)) * I_minus
    return ans

CALL BARRIER OPTION

In [36]:
def d1(S, K, B, r, q, sigma, T, t):
    return ( np.log(S / K) + (r - q + 0.5 * sigma ** 2) * (T - t) ) / ( sigma * np.sqrt(T - t) )

In [37]:
def d2(S, K, B, r, q, sigma, T, t):
    return ( np.log(S / K) + (r - q - 0.5 * sigma ** 2) * (T - t) ) / ( sigma * np.sqrt(T - t) )

In [38]:
def d3(S, K, B, r, q, sigma, T, t):
    return ( np.log(S / B) + (r - q + 0.5 * sigma ** 2) * (T - t) ) / ( sigma * np.sqrt(T - t) )

In [39]:
def d4(S, K, B, r, q, sigma, T, t):
    return ( np.log(S / B) + (r - q - 0.5 * sigma ** 2) * (T - t) ) / ( sigma * np.sqrt(T - t) )

In [40]:
def d5(S, K, B, r, q, sigma, T, t):
    return ( np.log(S / B) - (r - q - 0.5 * sigma ** 2) * (T - t) ) / ( sigma * np.sqrt(T - t) )

In [41]:
def d6(S, K, B, r, q, sigma, T, t):
    return ( np.log(S / B) - (r - q + 0.5 * sigma ** 2) * (T - t) ) / ( sigma * np.sqrt(T - t) )

In [42]:
def d7(S, K, B, r, q, sigma, T, t):
    return ( np.log((S * K) / B ** 2) - (r - q - 0.5 * sigma ** 2) * (T - t) ) / ( sigma * np.sqrt(T - t) )

In [43]:
def d8(S, K, B, r, q, sigma, T, t):
    return ( np.log((S * K) / B ** 2) - (r - q + 0.5 * sigma ** 2) * (T - t) ) / ( sigma * np.sqrt(T - t) )

In [44]:
def a(S, K, B, r, q, sigma, T, t):
    return (B / S) ** (-1 + 2 * (r - q) / (sigma ** 2))

In [45]:
def b(S, K, B, r, q, sigma, T, t):
    return (B / S) ** (1 + 2 * (r - q) / (sigma ** 2))

In [46]:
def get_price_call_ui(S, K, B, r, q, sigma, T, t):
    first_part = S * np.exp(-q * (T - t)) * (norm.cdf(d3(S, K, B, r, q, sigma, T, t)) + b(S, K, B, r, q, sigma, T, t) * (norm.cdf(d6(S, K, B, r, q, sigma, T, t)) - norm.cdf(d8(S, K, B, r, q, sigma, T, t))))
    second_part = K * np.exp(-r * (T - t)) * (norm.cdf(d4(S, K, B, r, q, sigma, T, t)) + a(S, K, B, r, q, sigma, T, t) * (norm.cdf(d5(S, K, B, r, q, sigma, T, t)) - norm.cdf(d7(S, K, B, r, q, sigma, T, t))))
    return first_part - second_part 

In [47]:
price_call_ui = get_price_call_ui(S, K, U, r, q, sigma, T, t)
price_call_ui

1.1891026179375501

TRINOMIAL TREE FORMULA

In [48]:
N = int(( np.log(U) - np.log(S) ) / ( sigma * np.sqrt(3 * dt) ) + 0.5)


u = np.exp(sigma * np.sqrt(3 * dt))
d = 1 / u
m = 1

p_d = - ( r - q - (sigma ** 2) / 2 ) * dt / ( 2 * np.log(u) ) + (sigma ** 2) * dt / ( 2 * np.log(u) ** 2 )
p_m = 1 - (sigma ** 2) * dt / (np.log(u) ** 2)
p_u = ( r - q - (sigma ** 2) / 2 ) * dt / ( 2 * np.log(u) ) + (sigma ** 2) * dt / ( 2 * np.log(u) ** 2 )

In [49]:
def get_S_u(S, n):
    data_S = pd.DataFrame(np.zeros((int(n / 2), n + 1)))
    data_S.iloc[0, 0] = S
    for j in range(int(n / 2)):
        for i in range(j, n + 1):
            data_S.iloc[j, i] = S * np.power(u, j) * np.power(m, i - j)
    return data_S

In [50]:
def get_S_d(S, n):
    data_S = pd.DataFrame(np.zeros((int(n / 2), n + 1)))
    data_S.iloc[0, 0] = S
    for j in range(int(n / 2)):
        for i in range(j, n + 1):
            data_S.iloc[j, i] = S * np.power(d, j) * np.power(m, i - j)
    return data_S

In [51]:
data_S_u = get_S_u(S, n)
data_S_d = get_S_d(S, n)

In [52]:
data_S = pd.concat([data_S_u[::-1], data_S_d], ignore_index=True)
data_S = data_S.drop(int(n / 2))
data_S.index = range(data_S.shape[0])

In [53]:
data_S

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,90.778744,90.778744,90.778744,90.778744,90.778744,90.778744,90.778744,90.778744,90.778744,90.778744
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,87.503852,87.503852,87.503852,87.503852,87.503852,87.503852,87.503852,87.503852,87.503852,87.503852
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,84.347104,84.347104,84.347104,84.347104,84.347104,84.347104,84.347104,84.347104,84.347104,84.347104
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,81.304236,81.304236,81.304236,81.304236,81.304236,81.304236,81.304236,81.304236,81.304236,81.304236
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,78.371142,78.371142,78.371142,78.371142,78.371142,78.371142,78.371142,78.371142,78.371142,78.371142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.870955,2.870955,2.870955,2.870955,2.870955,2.870955,2.870955,2.870955,2.870955,2.870955
95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.767383,2.767383,2.767383,2.767383,2.767383,2.767383,2.767383,2.767383,2.767383,2.767383
96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.667549,2.667549,2.667549,2.667549,2.667549,2.667549,2.667549,2.667549,2.667549,2.667549
97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.571315,2.571315,2.571315,2.571315,2.571315,2.571315,2.571315,2.571315,2.571315,2.571315


In [54]:
data_S = data_S.iloc[int(n / 2) - N - 1: int(n / 2) + N, : N + 1]
data_S.index = range(data_S.shape[0])
data_S.columns = range(data_S.shape[1])

In [55]:
data_S[data_S < 18].fillna(0)

,0,1,2,3,4,5
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.0,0.000000,0.000000,0.000000,17.374778,17.374778
2,0.0,0.000000,0.000000,16.747973,16.747973,16.747973
3,0.0,0.000000,16.143781,16.143781,16.143781,16.143781
4,0.0,15.561385,15.561385,15.561385,15.561385,15.561385
5,15.0,15.000000,15.000000,15.000000,15.000000,15.000000
6,0.0,14.458867,14.458867,14.458867,14.458867,14.458867
7,0.0,0.000000,13.937256,13.937256,13.937256,13.937256
8,0.0,0.000000,0.000000,13.434462,13.434462,13.434462
9,0.0,0.000000,0.000000,0.000000,12.949806,12.949806


In [56]:
data_S = data_S[data_S.columns[::-1]]
data_S.columns = data_S.columns[::-1]

In [57]:
def payoff_barrier(S, K, U):
    return max(S - K, 0)

In [58]:
payoff = []
for s in data_S[0]:
    payoff.append(payoff_barrier(s, K, U))

data_payoff = pd.DataFrame(np.zeros(data_S.shape))
data_payoff[0] = payoff

In [59]:
for j in range(1, data_S.shape[1]):
    for i in range(1, data_S.shape[0] - 1):
        up = data_payoff.iloc[i - 1, j - 1]
        mid = data_payoff.iloc[i, j - 1]
        down = data_payoff.iloc[i + 1, j - 1]
        data_payoff.iloc[i, j] = (p_u * up + p_m * mid + p_d * down) * np.exp(-r * dt)

In [60]:
data_payoff.iloc[int(data_payoff.shape[0] / 2), -1]

0.03569799492004593

Tashs

In [61]:
S_values = np.zeros((2 * n + 1, n + 1))
S_values[n][0] = S

for i in range(n+1):
    if i == 0:
        for j in range(1, n+1):
            S_values[n][j] = S_values[n][j-1] * m
    else:
        for j in range(i, n+1):
            S_values[n-i][j] = S_values[n-i+1][j-1] * u
            S_values[n+i][j] = S_values[n+i-1][j-1] * d


In [62]:
pd.DataFrame(S_values)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,591.277124
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,569.946484,569.946484
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,549.385358,549.385358,549.385358
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,529.565986,529.565986,529.565986,529.565986
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,510.461608,510.461608,510.461608,510.461608,510.461608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.440778,0.440778,0.440778,0.440778,0.440778
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.424876,0.424876,0.424876,0.424876
198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.409549,0.409549,0.409549
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.394774,0.394774


In [63]:
pu = ((r - sigma**2/2) * dt / (2 * np.log(u)) 
            + sigma**2 * dt / (2 * np.log(u)**2)
)
pd = (- (r - sigma**2/2) * dt / (2 * np.log(u)) 
        + sigma**2 * dt / (2 * np.log(u)**2)
)
pm = 1 - sigma**2 * dt / np.log(u)**2

print(f"Barrier: pu={pu}  pd={pd}  pm={pm}")


barrier_idx = S_values[S_values[:, n]>=U].shape[0] - 1

V_values = np.zeros((2 * n + 1, n+1))
V_values[:, n] = np.maximum((S_values[:, n] - K), 0)

for j in range(n-1, -1, -1):
    for i in range(j, -1, -1):
        if n-i == barrier_idx:
            V_values[n-i][j] = 0
        else:
            V_values[n-i][j] = np.exp(-r * dt) * (pu * V_values[n-i-1][j+1]
                            + pm * V_values[n-i][j+1]
                            + pd * V_values[n-i+1][j+1]
                            )
        if n+i == barrier_idx:
            V_values[n+i][j] = 0
        else:
            V_values[n+i][j] = np.exp(-r * dt) * (pu * V_values[n+i-1][j+1]
                            + pm * V_values[n+i][j+1]
                            + pd * V_values[n+i+1][j+1]
                            )

In [64]:
V_values = compute_trinomial_uo_option_price(n, sigma, r, dt, u, K, U, S_values, 1)

Barrier: pu=0.17176977029746482  pd=0.16156356303586827  pm=0.666666666666667


In [65]:
pd.DataFrame(V_values)

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,100
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,575.277124
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,553.956041,553.946484
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,533.404470,533.394917,533.385358
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,513.594648,513.585100,513.575546,513.565986
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,494.499819,494.490275,494.480725,494.471170,494.461608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
198,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000
199,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000


In [66]:
def compute_trinomial_option_price(r, sigma, dt, S_values, K, n, option_flag):
    pu = (r - sigma**2/2) * np.sqrt(dt / (12 * sigma**2)) + 1/6

    pd = - (r - sigma**2/2) * np.sqrt(dt / (12 * sigma**2)) + 1/6
    pm = 2/3

    print(f"General: pu={pu}  pd={pd}  pm={pm}")

    V_values = np.zeros((2 * n + 1, n+1))
    V_values[:, n] = np.maximum(option_flag * (S_values[:, n] - K), 0)
    
    for j in range(n-1, -1, -1):
        for i in range(j, -1, -1):
            V_values[n-i][j] = np.exp(-r * dt) * (pu * V_values[n-i-1][j+1]
                            + pm * V_values[n-i][j+1]
                            + pd * V_values[n-i+1][j+1]
                            )
            V_values[n+i][j] = np.exp(-r * dt) * (pu * V_values[n+i-1][j+1]
                            + pm * V_values[n+i][j+1]
                            + pd * V_values[n+i+1][j+1]
                            )
    return V_values


In [67]:
General_V_values = compute_trinomial_option_price(r, sigma, dt, S_values, K, n, 1)

print(General_V_values[n][0] - V_values[n][0])
General_V_values[n][0], V_values[n][0]

General: pu=0.17176977029746496  pd=0.16156356303586836  pm=0.6666666666666666
1.1844917839683642


(1.2361843357822506, 0.05169255181388638)